## API

In [2]:
import json
import requests
import pandas as pd
from bs4 import BeautifulSoup


In [5]:
# Genero inputs para poder hacer la request a la API de coindesk con los parametros adecuados
list_months = {
"january":"31", "february":"28","march":"31","april":"30","may":"31","june":"30",
"july":"31","august":"31","september":"30","october":"31","november":"30","december":"31"
}
meses = list(list_months.keys())
while 1: 
    while 1:
        try:
            s_year = int(input("Please introduce the start year in format YYYY (from 2011 to 2018): "))
        except: 
            print("The format of the year is not valid, try again ...")
        else:
            if s_year not in range (2011,2019):
                print("The year is not between 2011 - 2018, tray again")
            if s_year in range(2011,2019):
                break

    while 1:
        try:
            e_year = int(input("Please introduce the end year in format YYYY (from 2011 to 2018): "))
        except: 
            print("The format of the year is not valid, try again ...")
        else:
            if e_year not in range (2011,2019):
                print("The year is not between 2011 - 2018, tray again")
            if e_year in range(2011,2019):
                break
                
    if (s_year > e_year):
        print("The end year must be preceded by the start year,try again ...")
        continue        
            
    while 1:
        try:
            s_month = input("Please introduce the start month (january,february,etc...): ").lower()
        except: 
            print("The format of the month is not valid, try again ...")
        else:
            if s_month in list_months.keys():
                break

    while 1:
        try:
            e_month = input("Please introduce the end month (january,february,etc...): ").lower()
        except: 
            print("The format of the month is not valid, try again ...")
        else:
            if e_month in list_months.keys():
                break
                
    if (s_year == e_year) and ( (meses.index(s_month)) > (meses.index(e_month))):
        print("The end month must be preceded by the start month, try again ...")
        continue      
    else: 
        break 

Please introduce the start year in format YYYY (from 2011 to 2018): 2018
Please introduce the end year in format YYYY (from 2011 to 2018): 2018
Please introduce the start month (january,february,etc...): january
Please introduce the end month (january,february,etc...): march


In [6]:
# Construyo el request para la api en funcion de los parametros recibidos
url_inic = "https://api.coindesk.com/v1/bpi/historical/close.json"
if s_month in meses[9:]:
    start = str(s_year) + "-" + str(meses.index(s_month)+1) + "-" + "01"
else: 
    start = str(s_year) + "-" + "0" + str(meses.index(s_month)+1) + "-" + "01"

if e_month in meses[9:]:
    ends = str(e_year) + "-"+ str(meses.index(e_month)+1) + "-" +str(list_months[e_month])
else:
    ends = str(e_year) + "-"+ "0" + str(meses.index(e_month)+1) + "-" +str(list_months[e_month])
    

# Envio el request a la API y recibo la respuesta 
response = requests.get(url_inic + "?start=" + start + "&end=" + ends)
results = response.json()

# Transformo la respuesta en un DataFrame y lo limpio para quedarme con las columnas que me interesa 
df_ = pd.DataFrame(results)
df = df_[["bpi"]]
df.columns = ["Bitcoin Price (USD)"]
df = df[:-2]
df.index = pd.to_datetime(df.index)


# Calculo la media del precio para los años solicitados, tengo que hacer un request para cada año. 
avg_year = df
avg_year = df.groupby(df.index.year).mean()
avg_year.reset_index()
avg_year.columns = ["Avg yearly Bitcoin price (USD)"]
sel_years = list(set(avg_year.index))
medias ={}
for years in sel_years: 
    response_ = requests.get(url_inic + "?start={}-01-01&end={}-12-31".format(years,years))
    result_ = response_.json()
    df2 = pd.DataFrame(result_)
    df3 = df2[["bpi"]].mean()[0]
    medias[years] = df3


In [9]:
avg_year = pd.DataFrame(list(medias.items())) 
by_year = avg_year.rename(columns={0:"Year",1:"Yearly Avg. Bitcoin Price (USD)"}) 
by_year = by_year.sort_values(by=["Year"])


[(2018, 8121.143640522875)]

In [123]:
# Calculo el precio promedio mensual del BTC 
avg_month = df
avg_month["Month"] = [(mes[0:4] + "-" + mes[5:7]) for fecha in avg_month.index for mes in str(fecha).split()[0::2]]
avg_month_ = avg_month.groupby("Month").mean() 
avg = avg_month_.reset_index(level=0)
avg.columns = ["Month","Monthly Avg Bitcoin Price"]

# Exporto las tablas generadas en formato .csv 
data = avg.dropna().to_csv('Output/monthly_price.csv', index=False)
data2 = by_year.to_csv("Output/yearly_price.csv",index=False)


## Web Scrapping

,Country,Currency,Spot
0,USD,US dollar,1.1417
1,JPY,Japanese yen,128.89
2,BGN,Bulgarian lev,1.9558
3,CZK,Czech koruna,25.782
4,DKK,Danish krone,7.4603
5,GBP,Pound sterling,0.87870
6,HUF,Hungarian forint,321.62
7,PLN,Polish zloty,4.3202
8,RON,Romanian leu,4.6612
9,SEK,Swedish krona,10.3123


In [ ]:
A="A"
